In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [ ]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
x = df.drop('label', axis = 'columns')
y = df['label']
x

In [ ]:


sns.countplot(y, data = df)

In [ ]:
#Vocabulary size
voc_size = 5000

In [ ]:
messages = x.copy()
messages['title'][1]

In [ ]:
messages.reset_index(inplace=True)
messages

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[:10]

In [ ]:
onehot = [one_hot(words,voc_size) for words in corpus]
onehot[:10]

## Embedding Representation

In [ ]:
sent_length = 20
embeded_docs = pad_sequences(onehot, padding='pre',maxlen = sent_length)
embeded_docs[0]

## Creating the model

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
x_final = np.array(embeded_docs)
y_final = np.array(y)

In [ ]:
x_final.shape, y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.3, random_state=41)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test,y_test),epochs=10, batch_size=64)

In [ ]:
y_pred = (model.predict(x_test) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
